## Data importing & feature selection

In [1]:
#%%writefile "select-filter.py"
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif

# loading data to tables
file = open("heart.dat")
all = np.loadtxt(file, delimiter=" ")
X = np.zeros((len(all), len(all[0]) - 1), dtype=np.uint8)
y = np.zeros((len(all)), dtype=np.uint8)
for i in range(len(all)):
    for j in range(len(all[0]) - 1):
        X[i][j] = all[i][j]
    y[i] = all[i][len(all[0]) - 1]
    
features_quantity = len(X[0]-1)

# feature selection
featureSelection = SelectKBest(f_classif)
featureSelection.fit(X, y)

# sorting by relevance
toSort = np.zeros(
    (len(featureSelection.scores_)), dtype=([("key", "<i4"), ("val", "<f8")])
)
for i in range(len(featureSelection.scores_)):
    toSort[i]["key"] = i + 1
    toSort[i]["val"] = featureSelection.scores_[i]
toSort = np.sort(toSort, order="val")[::-1]

feature_order = [] 
for i in range(features_quantity):
    print("%d. Feature no. %d, Score: %f" % (i+1, toSort[i][0], toSort[i][1]))
    feature_order.append(toSort[i][0])
    



1. Feature no. 13, Score: 101.985062
2. Feature no. 12, Score: 70.098351
3. Feature no. 9, Score: 57.169632
4. Feature no. 8, Score: 56.908981
5. Feature no. 3, Score: 56.554451
6. Feature no. 10, Score: 54.564568
7. Feature no. 11, Score: 34.477673
8. Feature no. 2, Score: 26.065310
9. Feature no. 1, Score: 12.651622
10. Feature no. 7, Score: 9.190846
11. Feature no. 4, Score: 6.630479
12. Feature no. 5, Score: 5.024383
13. Feature no. 6, Score: 0.071390


## Sorting the dataset
For convenience reasons, the dataset columns can be sorted by relevance.

In [2]:
def complement(arr):
    new_arr = []
    for x in range(len(arr)):
        new_arr.append(abs(arr[x]-len(arr)))
    return np.flip(new_arr)

permutation = complement(feature_order)

idx = np.empty_like(permutation)
idx[permutation] = np.arange(len(permutation))
X = X[:, idx]
print(X)



[[  3   3   2 ...   0   0  66]
 [  7   0   2 ...   0   0  52]
 [  7   0   0 ...   0   0   5]
 ...
 [  3   0   2 ...   0   0  38]
 [  6   0   0 ...   0   0 192]
 [  3   3   2 ...   1   0  30]]


Now the first column is the most relevant feature, the second is the second most important and so on.

## Diving data into test set and training set

### 2-fold cross validation

For validation, 2-fold cross validation repated 5 times is used.



In [3]:
from sklearn.model_selection import RepeatedKFold

rkf = RepeatedKFold(n_splits=2, n_repeats=5, random_state=10)


## KNN Alghoritm

Chosen k-values are 5, 7, 9 (odd values, because amount of classes is even).
Chosen metrics are:
- Euclidean distance
- Manhattan distance

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

metric_list = ['manhattan', 'euclidean']
max_features = 6

highest_acc = 0
lowest_acc = 1

# print(X_train[:,:2])
for top in range(max_features):
    print("\n")
    features_train = X[:,:(top+1)]
    acc_list = []
    for p in metric_list:
        for k in range(5, 10, 2):
            model = KNeighborsClassifier(n_neighbors=k, metric=p)
            scores = cross_val_score(model, features_train, y, scoring='accuracy', cv=rkf, n_jobs=-1)
            result = mean(scores)
            print('Accuracy(k=%d, %s metric, top %d features): %.3f (%.3f)' % (k, p, top+1, result, std(scores)))
            acc_list.append(result)
            
            if result > highest_acc:
                highest_acc = result
            if result < lowest_acc:
                lowest_acc = result
    print('Mean accuracy for top %d features: %f' % (top+1, mean(acc_list)))

print("\nMaximum accuracy:", highest_acc)
print("Minimum accuracy:", lowest_acc)





Accuracy(k=5, manhattan metric, top 1 features): 0.691 (0.127)
Accuracy(k=7, manhattan metric, top 1 features): 0.699 (0.116)
Accuracy(k=9, manhattan metric, top 1 features): 0.666 (0.187)
Accuracy(k=5, euclidean metric, top 1 features): 0.691 (0.127)
Accuracy(k=7, euclidean metric, top 1 features): 0.699 (0.116)
Accuracy(k=9, euclidean metric, top 1 features): 0.666 (0.187)
Mean accuracy for top 1 features: 0.685432


Accuracy(k=5, manhattan metric, top 2 features): 0.754 (0.019)
Accuracy(k=7, manhattan metric, top 2 features): 0.759 (0.029)
Accuracy(k=9, manhattan metric, top 2 features): 0.753 (0.027)
Accuracy(k=5, euclidean metric, top 2 features): 0.754 (0.019)
Accuracy(k=7, euclidean metric, top 2 features): 0.759 (0.029)
Accuracy(k=9, euclidean metric, top 2 features): 0.753 (0.027)
Mean accuracy for top 2 features: 0.755309


Accuracy(k=5, manhattan metric, top 3 features): 0.770 (0.039)
Accuracy(k=7, manhattan metric, top 3 features): 0.776 (0.026)
Accuracy(k=9, manhattan me

## Statistical analysis

TBD